# Panel Regression - Firm Characteristics

A panel regression is a suitable regression method for analyzing our data, especially since you have quarterly data for multiple companies over a long period of time. A panel regression model accounts for both within-entity and between-entity variations, making it a useful tool to analyze data with a time series and cross-sectional dimension.  
With panel data, we can control for individual-level characteristics that may affect forecast accuracy by including fixed effects for each company or industry. We can also account for time-specific factors that may affect forecast accuracy by including time fixed effects or time-varying covariates.

### Random Effects Panel Regression

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects


Data

In [3]:
df = pd.read_csv("Dataframes/characteristics.csv")
df

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Revenue - Actual,Enterprise Value,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,AVY.N,2013-01-01,Materials,11.178,11.178,1.532200e+09,4.413001e+09,3.426001e+09,0.799347,10.637479,207295.0,-1
1,AVY.N,2013-04-01,Materials,2.482,2.482,1.498900e+09,5.459128e+09,4.309428e+09,0.895787,24.148726,0.0,-1
2,AVY.N,2013-07-01,Materials,1.068,1.068,1.552300e+09,5.436229e+09,4.258229e+09,0.906078,-0.054218,0.0,-1
3,AVY.N,2013-10-01,Materials,8.095,8.095,1.504900e+09,5.029514e+09,4.273414e+09,0.903176,2.467620,236291.0,-1
4,AVY.N,2014-01-01,Materials,1.471,1.471,1.583900e+09,5.553068e+09,4.877168e+09,0.920659,16.003852,157761.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
19299,POOL.OQ,2021-10-01,Retailing,17.194,17.194,1.411448e+09,1.769954e+10,1.742020e+10,3.684400,-5.130322,157740.0,-1
19300,POOL.OQ,2022-01-01,Retailing,40.267,40.267,1.035557e+09,2.384882e+10,2.268979e+10,4.504382,30.489705,82349.0,-1
19301,POOL.OQ,2022-04-01,Retailing,34.342,34.342,1.412650e+09,1.843682e+10,1.696712e+10,3.422880,-25.160664,290897.0,-1
19302,POOL.OQ,2022-07-01,Retailing,1.503,1.503,2.055818e+09,1.556187e+10,1.405795e+10,2.749481,-21.661570,185584.0,-1


In [4]:
df["Date"] = pd.to_datetime(df["Date"])

Remove outliers

In [5]:
#removing outliers
summary_stats = df["Earnings Per Share - Actual Surprise"].describe()
Q1 = summary_stats.loc['25%']
Q3 = summary_stats.loc['75%']
IQR = Q3 - Q1
threshold = 7
surprise_outliers_removed = df["Earnings Per Share - Actual Surprise"].loc[~((df["Earnings Per Share - Actual Surprise"] < (Q1 - threshold * IQR)) | (df["Earnings Per Share - Actual Surprise"] > (Q3 + threshold * IQR)))]
df_clean = df.copy()
df_clean["Earnings Per Share - Actual Surprise"] = surprise_outliers_removed
df_clean = df_clean.dropna(subset=["Earnings Per Share - Actual Surprise"])
#df_clean = df_clean.dropna()
df_clean

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Revenue - Actual,Enterprise Value,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,AVY.N,2013-01-01,Materials,11.178,11.178,1.532200e+09,4.413001e+09,3.426001e+09,0.799347,10.637479,207295.0,-1
1,AVY.N,2013-04-01,Materials,2.482,2.482,1.498900e+09,5.459128e+09,4.309428e+09,0.895787,24.148726,0.0,-1
2,AVY.N,2013-07-01,Materials,1.068,1.068,1.552300e+09,5.436229e+09,4.258229e+09,0.906078,-0.054218,0.0,-1
3,AVY.N,2013-10-01,Materials,8.095,8.095,1.504900e+09,5.029514e+09,4.273414e+09,0.903176,2.467620,236291.0,-1
4,AVY.N,2014-01-01,Materials,1.471,1.471,1.583900e+09,5.553068e+09,4.877168e+09,0.920659,16.003852,157761.0,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
19299,POOL.OQ,2021-10-01,Retailing,17.194,17.194,1.411448e+09,1.769954e+10,1.742020e+10,3.684400,-5.130322,157740.0,-1
19300,POOL.OQ,2022-01-01,Retailing,40.267,40.267,1.035557e+09,2.384882e+10,2.268979e+10,4.504382,30.489705,82349.0,-1
19301,POOL.OQ,2022-04-01,Retailing,34.342,34.342,1.412650e+09,1.843682e+10,1.696712e+10,3.422880,-25.160664,290897.0,-1
19302,POOL.OQ,2022-07-01,Retailing,1.503,1.503,2.055818e+09,1.556187e+10,1.405795e+10,2.749481,-21.661570,185584.0,-1


In [ ]:
#other way to deal with outlierrs: removing only 99th percentile outliers
'''
# Calculate the 99th percentile
pct_99 = np.percentile(df['Earnings Per Share - Actual Surprise'], 99)

# Remove the data points above the 99th percentile
df_clean = df[df['Earnings Per Share - Actual Surprise'] <= pct_99].copy()

df_clean
'''

Standardising some of the independent variables

In [6]:
rescale = df_clean
#rescale["Earnings Per Share - Actual Surprise"] = MinMaxScaler().fit_transform(np.array(rescale["Earnings Per Share - Actual Surprise"]).reshape(-1,1))
rescale["Revenue - Actual"] = MinMaxScaler().fit_transform(np.array(rescale["Revenue - Actual"]).reshape(-1,1))
rescale["Enterprise Value"] = MinMaxScaler().fit_transform(np.array(rescale["Enterprise Value"]).reshape(-1,1))
rescale["Market Capitalization"] = MinMaxScaler().fit_transform(np.array(rescale["Market Capitalization"]).reshape(-1,1))
rescale["Enterprise Value To Sales (Daily Time Series Ratio)"] = MinMaxScaler().fit_transform(np.array(rescale["Enterprise Value To Sales (Daily Time Series Ratio)"]).reshape(-1,1))
rescale["3 Month Total Return"] = MinMaxScaler().fit_transform(np.array(rescale["3 Month Total Return"]).reshape(-1,1))
rescale["Volume"] = MinMaxScaler().fit_transform(np.array(rescale["Volume"]).reshape(-1,1))
df_clean = rescale
df_clean

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,Revenue - Actual,Enterprise Value,Market Capitalization,Enterprise Value To Sales (Daily Time Series Ratio),3 Month Total Return,Volume,loss firm status
0,AVY.N,2013-01-01,Materials,11.178,11.178,0.065123,0.027862,0.001166,0.027195,0.298034,0.001183,-1
1,AVY.N,2013-04-01,Materials,2.482,2.482,0.064917,0.028207,0.001471,0.027308,0.340793,0.000000,-1
2,AVY.N,2013-07-01,Materials,1.068,1.068,0.065247,0.028199,0.001453,0.027321,0.264198,0.000000,-1
3,AVY.N,2013-10-01,Materials,8.095,8.095,0.064955,0.028065,0.001459,0.027317,0.272179,0.001348,-1
4,AVY.N,2014-01-01,Materials,1.471,1.471,0.065443,0.028238,0.001667,0.027338,0.315017,0.000900,-1
...,...,...,...,...,...,...,...,...,...,...,...,...
19299,POOL.OQ,2021-10-01,Retailing,17.194,17.194,0.064377,0.032245,0.006004,0.030598,0.248134,0.000900,-1
19300,POOL.OQ,2022-01-01,Retailing,40.267,40.267,0.062055,0.034274,0.007826,0.031566,0.360861,0.000470,-1
19301,POOL.OQ,2022-04-01,Retailing,34.342,34.342,0.064385,0.032488,0.005848,0.030290,0.184744,0.001660,-1
19302,POOL.OQ,2022-07-01,Retailing,1.503,1.503,0.068358,0.031540,0.004842,0.029495,0.195817,0.001059,-1


In [7]:
df_panel = df_clean.copy()

#### Fixed Effects Panel Regression

In a fixed effects panel regression, the individual-specific effects are modeled as fixed variables that do not vary across time. This means that the coefficients of the independent variables are estimated based on the within-entity variation in the data, which eliminates the effect of time-invariant unobserved heterogeneity.

Fixed effects models are useful when there are time-invariant unobserved variables that may affect the dependent variable, but are not included in the model. By modeling the individual-specific effects as fixed variables, fixed effects models can control for this unobserved heterogeneity and estimate the coefficients of the independent variables based on the within-entity variation, which provides more efficient estimates of the coefficients.

One limitation of fixed effects models is that they do not allow for testing the effect of time-invariant variables on the dependent variable. In addition, fixed effects models may suffer from the incidental parameter problem, which may lead to biased estimates of the coefficients of the independent variables in the presence of a large number of fixed effects.

In [8]:
#reformatting indices of dataframe for panel regression
df_clean['Instrument'] = df_clean['Instrument'].astype('category')
df_clean['Date'] = pd.to_datetime(df_clean['Date'])

# set the index to be the time variable and the cross-sectional variable
df_clean.set_index(['Instrument', 'Date'], inplace=True)
df_clean

GICS Industry Group Name  \
Instrument Date                                  
AVY.N      2013-01-01                Materials   
           2013-04-01                Materials   
           2013-07-01                Materials   
           2013-10-01                Materials   
           2014-01-01                Materials   
...                                        ...   
POOL.OQ    2021-10-01                Retailing   
           2022-01-01                Retailing   
           2022-04-01                Retailing   
           2022-07-01                Retailing   
           2022-10-01                Retailing   

                       Earnings Per Share - Actual Surprise  \
Instrument Date                                               
AVY.N      2013-01-01                                11.178   
           2013-04-01                                 2.482   
           2013-07-01                                 1.068   
           2013-10-01                                 8.095   
           2014-01-01                                 1.471   
...                                                     ...   
POOL.OQ    2021-10-01                                17.194   
           2022-01-01                                40.267   
           2022-04-01                                34.342   
           2022-07-01                                 1.503   
           2022-10-01                                 4.196   

                       Earnings Per Share - Actual Surprise AbsVals  \
Instrument Date                                                       
AVY.N      2013-01-01                                        11.178   
           2013-04-01                                         2.482   
           2013-07-01                                         1.068   
           2013-10-01                                         8.095   
           2014-01-01                                         1.471   
...                                                             ...   
POOL.OQ    2021-10-01                                        17.194   
           2022-01-01                                        40.267   
           2022-04-01                                        34.342   
           2022-07-01                                         1.503   
           2022-10-01                                         4.196   

                       Revenue - Actual  Enterprise Value  \
Instrument Date                                             
AVY.N      2013-01-01          0.065123          0.027862   
           2013-04-01          0.064917          0.028207   
           2013-07-01          0.065247          0.028199   
           2013-10-01          0.064955          0.028065   
           2014-01-01          0.065443          0.028238   
...                                 ...               ...   
POOL.OQ    2021-10-01          0.064377          0.032245   
           2022-01-01          0.062055          0.034274   
           2022-04-01          0.064385          0.032488   
           2022-07-01          0.068358          0.031540   
           2022-10-01          0.065637          0.031045   

                       Market Capitalization  \
Instrument Date                                
AVY.N      2013-01-01               0.001166   
           2013-04-01               0.001471   
           2013-07-01               0.001453   
           2013-10-01               0.001459   
           2014-01-01               0.001667   
...                                      ...   
POOL.OQ    2021-10-01               0.006004   
           2022-01-01               0.007826   
           2022-04-01               0.005848   
           2022-07-01               0.004842   
           2022-10-01               0.004337   

                       Enterprise Value To Sales (Daily Time Series Ratio)  \
Instrument Date                                                              
AVY.N      2013-01-01                               

Regression with **Actual** Suprise Values

In [9]:
y = df_clean.loc[:, "Earnings Per Share - Actual Surprise"]
X = df_clean.loc[:, "Revenue - Actual":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


Dep. Variable:,Earnings Per Share - Actual Surprise,R-squared:,0.0080
Estimator:,PanelOLS,R-squared (Between):,0.5637
No. Observations:,18533,R-squared (Within):,0.0125
Date:,"Thu, Mar 02 2023",R-squared (Overall):,0.1451
Time:,11:06:48,Log-likelihood,-7.746e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,20.832
Entities:,501,P-value,0.0000
Avg Obs:,36.992,Distribution:,"F(7,17985)"
Min Obs:,6.0000,,
Max Obs:,40.000,F-statistic (robust):,20.832


Regression with **Absolute** Surprise Values

In [10]:
y = df_clean.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = df_clean.loc[:, "Revenue - Actual":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0070
Estimator:,PanelOLS,R-squared (Between):,-0.3487
No. Observations:,18533,R-squared (Within):,0.0098
Date:,"Thu, Mar 02 2023",R-squared (Overall):,-0.1982
Time:,11:07:07,Log-likelihood,-7.248e+04
Cov. Estimator:,Unadjusted,,
,,F-statistic:,18.101
Entities:,501,P-value,0.0000
Avg Obs:,36.992,Distribution:,"F(7,17985)"
Min Obs:,6.0000,,
Max Obs:,40.000,F-statistic (robust):,18.101


#### By Industry: Example Tech Industry

Actual Surprise Values

In [11]:
group = df_panel[df_panel['GICS Industry Group Name'] == 'Technology Hardware & Equipment']

group['Instrument'] = group['Instrument'].astype('category')
group['Date'] = pd.to_datetime(group['Date'])
group.set_index(['Instrument', 'Date'], inplace=True)

y = group.loc[:, "Earnings Per Share - Actual Surprise"]
X = group.loc[:, "Revenue - Actual":]

model = PanelOLS(y, X, entity_effects=True, time_effects=True).fit()
model

/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_94708/995584927.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Instrument'] = group['Instrument'].astype('category')
/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_94708/995584927.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Date'] = pd.to_datetime(group['Date'])


Dep. Variable:,Earnings Per Share - Actual Surprise,R-squared:,0.0117
Estimator:,PanelOLS,R-squared (Between):,-1.0519
No. Observations:,727,R-squared (Within):,0.0250
Date:,"Thu, Mar 02 2023",R-squared (Overall):,-0.4032
Time:,18:50:11,Log-likelihood,-2690.9
Cov. Estimator:,Unadjusted,,
,,F-statistic:,1.1157
Entities:,19,P-value,0.3512
Avg Obs:,38.263,Distribution:,"F(7,661)"
Min Obs:,27.000,,
Max Obs:,40.000,F-statistic (robust):,1.1157


Absolute Surprise Values

In [12]:
group = df_panel[df_panel['GICS Industry Group Name'] == 'Technology Hardware & Equipment']

group['Instrument'] = group['Instrument'].astype('category')
group['Date'] = pd.to_datetime(group['Date'])
group.set_index(['Instrument', 'Date'], inplace=True)

y = group.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = group.loc[:, "Revenue - Actual":]

model = PanelOLS(y, X, entity_effects=True, time_effects=True).fit()
model

/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_94708/3204827800.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Instrument'] = group['Instrument'].astype('category')
/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_94708/3204827800.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Date'] = pd.to_datetime(group['Date'])


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0277
Estimator:,PanelOLS,R-squared (Between):,-2.4456
No. Observations:,727,R-squared (Within):,0.0319
Date:,"Thu, Mar 02 2023",R-squared (Overall):,-1.3166
Time:,18:51:16,Log-likelihood,-2597.1
Cov. Estimator:,Unadjusted,,
,,F-statistic:,2.6870
Entities:,19,P-value,0.0095
Avg Obs:,38.263,Distribution:,"F(7,661)"
Min Obs:,27.000,,
Max Obs:,40.000,F-statistic (robust):,2.6870
